In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
import torch
from transformers import TrainingArguments,Trainer
from transformers import BertForSequenceClassification,BertTokenizer

In [2]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

one_hot=OneHotEncoder(sparse=False)

In [3]:
train=pd.read_csv("./Corona_NLP_train.csv",encoding='latin-1')
test=pd.read_csv("./Corona_NLP_test.csv",encoding='latin-1')
train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
train=train[:5000]

In [5]:
train=pd.get_dummies(train,columns=['Sentiment'])

In [6]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
model=model.to("cuda")

In [8]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [9]:
sample_data=["I am eating apple","I am playing cricket"]
tokenizer(sample_data,truncation=True,padding=True,return_tensors='pt')

{'input_ids': tensor([[ 101, 1045, 2572, 5983, 6207,  102],
        [ 101, 1045, 2572, 2652, 4533,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]])}

In [10]:
from sklearn.model_selection import train_test_split

X=train['OriginalTweet'].to_list()
Y=train['Sentiment_Extremely Negative'].to_list()
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42,stratify=Y)
x_train_tokenized=tokenizer(x_train,padding=True,truncation=True,max_length=512)#rest will be padded
x_test_tokenized=tokenizer(x_test,padding=True,truncation=True,max_length=512)#we set truncation to true if we want to cut down text to max length if text
#is longer than 512 tokens

In [11]:
x_train[0]

'As the #coronavirus makes an impact around the world, many consumers are shopping online to purchase the items that they need. Here are 3 things that #eCommerce businesses can do to prepare for the coronavirus pandemic.\r\r\n\r\r\nhttps://t.co/rcULq8vDOS'

In [12]:
x_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [13]:
x_train_tokenized['input_ids'][0]

[101,
 2004,
 1996,
 1001,
 21887,
 23350,
 3084,
 2019,
 4254,
 2105,
 1996,
 2088,
 1010,
 2116,
 10390,
 2024,
 6023,
 3784,
 2000,
 5309,
 1996,
 5167,
 2008,
 2027,
 2342,
 1012,
 2182,
 2024,
 1017,
 2477,
 2008,
 1001,
 17338,
 15810,
 3401,
 5661,
 2064,
 2079,
 2000,
 7374,
 2005,
 1996,
 21887,
 23350,
 6090,
 3207,
 7712,
 1012,
 16770,
 1024,
 1013,
 1013,
 1056,
 1012,
 2522,
 1013,
 22110,
 5313,
 4160,
 2620,
 16872,
 2891,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [14]:
x_train_tokenized['token_type_ids'][0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [15]:
x_train_tokenized['attention_mask'][0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

in the getitem function note that we are iterating for the tokenizer section such that for example key is input id and value is the encoded text part that is 101...102 and we are iterating through each text by using for loop 

In [17]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,encodings,labels=None):#so sometimes u may not have labels so im init ing it to None
        self.encodings=encodings
        self.labels=labels
        
    def __getitem__(self,idx):
        item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}#obtaining in a key-value format as in the tokenizer part
        if self.labels:
            item["labels"]=torch.tensor(self.labels[idx])# if label is present we iterate over it.
        return item
    
    def __len__(self):
        return len(self.encodings['input_ids'])#return the length of the dataset..Note: here the encodings are outputs from the tokenizer

Now all the values are tensors of type torch.tensors

In [18]:
train_dataset=Dataset(x_train_tokenized,y_train)
test_dataset=Dataset(x_test_tokenized,y_test)#where X-test_tokenized is encodings and y_test is the label

In [19]:
train_dataset[5]

{'input_ids': tensor([  101,  2065,  2045,  1005,  1055,  2019,  9750,  2711,  1013,  3232,
          2008,  2064,  2102,  2191,  2009,  2000,  1996, 13025,  3573,  2030,
          3432,  2987,  1005,  1056,  2215,  2000,  2175,  1010,  1045,  1005,
          1040,  2022,  3407,  2000,  2175,  2005,  2017,  1012,  2065,  2017,
          2113,  2619,  1999,  2342,  1999,  2030,  2105,  4572,  3077,  1013,
         10424,  2483,  3597,  3531,  1040,  2213,  2033,  1998,  2092,  3046,
          2000,  3275,  2023,  2041,  1012,  1001, 21887, 23350,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [20]:
train_dataset[5]['input_ids']

tensor([  101,  2065,  2045,  1005,  1055,  2019,  9750,  2711,  1013,  3232,
         2008,  2064,  2102,  2191,  2009,  2000,  1996, 13025,  3573,  2030,
         3432,  2987,  1005,  1056,  2215,  2000,  2175,  1010,  1045,  1005,
         1040,  2022,  3407,  2000,  2175,  2005,  2017,  1012,  2065,  2017,
         2113,  2619,  1999,  2342,  1999,  2030,  2105,  4572,  3077,  1013,
        10424,  2483,  3597,  3531,  1040,  2213,  2033,  1998,  2092,  3046,
         2000,  3275,  2023,  2041,  1012,  1001, 21887, 23350,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [21]:
train_dataset[5]['token_type_ids']

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0])

In [22]:
train_dataset[5]['attention_mask']

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0])

In [23]:
def compute_metrics(p):
    pred,labels=p
    pred=np.argmax(pred,axis=1)
    
    accuracy=accuracy_score(y_true=labels,y_pred=pred)
    precision=precision_score(y_true=labels,y_pred=pred)
    recall=recall_score(y_true=labels,y_pred=pred)
    score_f1=f1_score(y_true=labels,y_pred=pred)
    return {"Accuracy":accuracy,"Precision Score":precision,"Recall":recall,'f1_score':score_f1}

In [24]:
#define trainer
args=TrainingArguments(
    output_dir="outputs",
    num_train_epochs=1,
    per_device_train_batch_size=8
)

trainer=Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

c:\Anaconda\envs\torch_env\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3750
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 469
  Number of trainable parameters = 109483778
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rajashekarvt58 (team-torch). Use `wandb login --relogin` to force relogin


  0%|          | 0/469 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 110.6683, 'train_samples_per_second': 33.885, 'train_steps_per_second': 4.238, 'train_loss': 0.4348703152589452, 'epoch': 1.0}


TrainOutput(global_step=469, training_loss=0.4348703152589452, metrics={'train_runtime': 110.6683, 'train_samples_per_second': 33.885, 'train_steps_per_second': 4.238, 'train_loss': 0.4348703152589452, 'epoch': 1.0})

In [26]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1250
  Batch size = 8


  0%|          | 0/157 [00:00<?, ?it/s]

C:\Users\rajashekar V.T\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.3842964768409729,
 'eval_Accuracy': 0.8464,
 'eval_Precision Score': 0.0,
 'eval_Recall': 0.0,
 'eval_f1_score': 0.0,
 'eval_runtime': 4.8718,
 'eval_samples_per_second': 256.579,
 'eval_steps_per_second': 32.226,
 'epoch': 1.0}

In [27]:
text="Valorant is a better game than fortnite? its debatable"
inputs=tokenizer(text,truncation=True,padding=True,return_tensors='pt').to('cuda')
outputs=model(**inputs)
print(outputs)
predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
print(predictions)
predictions=predictions.cpu().detach().cpu()
print(predictions)

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.3312, -0.6090]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.8744, 0.1256]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([[0.8744, 0.1256]])


In [28]:
trainer.save_model("CustomModel")

Saving model checkpoint to CustomModel
Configuration saved in CustomModel\config.json
Model weights saved in CustomModel\pytorch_model.bin


In [29]:
model_2=BertForSequenceClassification.from_pretrained('CustomModel')
model_2.to("cuda")

loading configuration file CustomModel\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file CustomModel\pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were ini

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [30]:
text="Valorant is awesome"
inputs=tokenizer(text,truncation=True,padding=True,return_tensors='pt').to('cuda')
outputs=model_2(**inputs)
print(outputs)
predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
print(predictions)
predictions=predictions.cpu().detach().cpu()
print(predictions)

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.6008, -0.9865]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9300, 0.0700]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([[0.9300, 0.0700]])


In [33]:
import huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\rajashekar V.T\.huggingface\token
Login successful


In [34]:
model_2.push_to_hub('rajashekarvt/FineTunedBert')

Configuration saved in C:\Users\RAJASH~1.T\AppData\Local\Temp\tmprkehizz9\config.json
Model weights saved in C:\Users\RAJASH~1.T\AppData\Local\Temp\tmprkehizz9\pytorch_model.bin
Uploading the following files to rajashekarvt/FineTunedBert: config.json,pytorch_model.bin


CommitInfo(commit_url='https://huggingface.co/rajashekarvt/FineTunedBert/commit/a54b166f357f428ecb32f58041c694703c307aa9', commit_message='Upload BertForSequenceClassification', commit_description='', oid='a54b166f357f428ecb32f58041c694703c307aa9', pr_url=None, pr_revision=None, pr_num=None)